In [6]:
import os
import cv2
import math
import numpy as np
import cv2
import numpy as np
from sklearn.cluster import KMeans

cwd = os.getcwd()
cwd

'd:\\cv_project'

In [3]:
def FilterLines(Lines):
    REJECT_DEGREE_TH = 4.0

    FinalLines = []
    
    for Line in Lines:
        [[x1, y1, x2, y2]] = Line

        # Calculating equation of the line: y = mx + c
        if x1 != x2:
            m = (y2 - y1) / (x2 - x1)
        else:
            m = 100000000

        c = y2 - m*x2
        # theta will contain values between -90 -> +90. 
        theta = math.degrees(math.atan(m))

        # Rejecting lines of slope near to 0 degree or 90 degree and storing others
        if REJECT_DEGREE_TH <= abs(theta) <= (90 - REJECT_DEGREE_TH):
            l = math.sqrt( (y2 - y1)**2 + (x2 - x1)**2 )    # length of the line
            FinalLines.append([x1, y1, x2, y2, m, c, l])

    
    # Removing extra lines 
    # (we might get many lines, so we are going to take only longest 15 lines 
    # for further computation because more than this number of lines will only 
    # contribute towards slowing down of our algo.)
    if len(FinalLines) > 15:
        FinalLines = sorted(FinalLines, key=lambda x: x[-1], reverse=True)
        FinalLines = FinalLines[:15]
    
    return FinalLines


def GetLines(Image):
    # Converting to grayscale
    GrayImage = cv2.cvtColor(Image, cv2.COLOR_BGR2GRAY)
    # Blurring image to reduce noise.
    BlurGrayImage = cv2.GaussianBlur(GrayImage, (5, 5), 1)
    # Generating Edge image
    EdgeImage = cv2.Canny(BlurGrayImage, 40, 255)

    # Finding Lines in the i    mage
    Lines = cv2.HoughLinesP(EdgeImage, 1, np.pi / 180, 50, 10, 15)

    # Check if lines found and exit if not.
    if Lines is None:
        print("Not enough lines found in the image for Vanishing Point detection.")
        exit(0)
    
    # Filtering Lines wrt angle
    FilteredLines = FilterLines(Lines)

    return FilteredLines

def getCordinates(m1 ,c1, m2, c2, shape, y):
    height, width, _ = shape
    y = int(y+10)

    # y =x+c
    if m1 > m2:
        m1, m2 = m2, m1
        c1, c2 = c2, c1


    x10 = (y - c1)/(m1)
    x11 = (height - 1 - c1)/m1

    x20 = (y - c2) /m2
    x21 = (height  -1 - c2) /m2

    x10 = int(x10)
    x11 = int(x11)
    x20 = int(x20)
    x21 = int(x21)

    
    return (x10, y), (x11, height-1), (x20, y), (x21, height-1)
    

def GetVanishingPoint(Lines, shape):
    # We will apply RANSAC inspired algorithm for this. We will take combination 
    # of 2 lines one by one, find their intersection point, and calculate the 
    # total error(loss) of that point. Error of the point means root of sum of 
    # squares of distance of that point from each line.
    VanishingPoint = None
    MinError = 100000000000
    m1_best, c1_best = 0, 0
    m2_best, c2_best = 0, 0

    for i in range(len(Lines)):
        for j in range(i+1, len(Lines)):
            m1, c1 = Lines[i][4], Lines[i][5]
            m2, c2 = Lines[j][4], Lines[j][5]

            if m1 != m2:
                x0 = (c1 - c2) / (m2 - m1)
                y0 = m1 * x0 + c1

                err = 0
                for k in range(len(Lines)):
                    m, c = Lines[k][4], Lines[k][5]
                    m_ = (-1 / m)
                    c_ = y0 - m_ * x0

                    x_ = (c - c_) / (m_ - m)
                    y_ = m_ * x_ + c_

                    l = math.sqrt((y_ - y0)**2 + (x_ - x0)**2)

                    err += l**2

                err = math.sqrt(err)

                if MinError > err:
                    MinError = err
                    VanishingPoint = [x0, y0]
                    m1_best, c1_best = m1, c1
                    m2_best, c2_best = m2, c2

    if VanishingPoint is None:
        return VanishingPoint, (0, 0, 0, 0)

    if m1_best*m2_best > 0:
        return None, (0, 0, 0, 0)

    v11, v12, v21, v22 = getCordinates(m1_best ,c1_best, m2_best, c2_best, shape, VanishingPoint[1])
    # v11, v12, v21, v22 = None, None, None, None
    # print(v11, v12, v21, v22)

    return VanishingPoint, ( v11, v12, v21, v22)

In [4]:
def getPoints(homography, data):
    data = np.append(data, np.ones((len(data), 1)), axis=1)

    tr_data = (homography @ data.T).T

    for i in range(tr_data.shape[1]):
        tr_data[:, i] = tr_data[:,i]/ tr_data[:,tr_data.shape[1]-1]

    return tr_data

K_matrix = np.array([
    [0.8, 0, 50],
    [0, 1.2, 600],
    [0, 0, 1]
])

In [5]:
import cv2
import numpy as np
import copy

# Load the video
cap = cv2.VideoCapture('D:\cv_project\cv_videos\\autobahn_360.mp4')

i = 0
while(cap.isOpened()):
    ret, frame = cap.read()
    if not ret:
        break

    try:
        Lines = GetLines(frame)
    except Exception as e:
        continue
    
    VanishingPoint, vertexes = GetVanishingPoint(Lines, frame.shape)

    if VanishingPoint is None:
        print("Vanishing Point not found. Possible reason is that not enough lines are found in the image for determination of vanishing point.")
        continue

    Image = copy.deepcopy(frame)

    for Line in Lines:
        cv2.line(Image, (Line[0], Line[1]), (Line[2], Line[3]), (0, 255, 0), 1)
    for vertex in vertexes:
        cv2.circle(Image, vertex, 5, (255, 0, 255), -1)

    cv2.circle(Image, (int(VanishingPoint[0]), int(VanishingPoint[1])), 5, (0, 0, 255), -1)
    
    cv2.line(Image, vertexes[0], vertexes[1], (255, 0, 0), 3)
    cv2.line(Image, vertexes[2], vertexes[3], (255, 0, 0), 3)

    src_points = [
        vertexes[0],
        vertexes[1],
        vertexes[3],
        vertexes[2],
    ]


    dst_points = [
        [0, -360],
        [0, 360],
        [640, 360],
        [640, -360],
    ]


    src_points = np.array(src_points)
    dst_points = np.array(dst_points)

    M1, mask1 = cv2.findHomography(src_points, dst_points, cv2.RANSAC, 0.05, maxIters=100000)
    
    height, width, d = Image.shape
    vertex = np.float32([
        [0 , 0],
        [0, height-1],
        [width-1, 0],
        [width-1, height-1],
    ])


    try:
        pts1 = getPoints(M1, vertex)
    except Exception as e:
        continue
    
    pts1 = pts1.astype(np.float32)

    pts1 = (pts1 @ K_matrix.T)[:, :2].astype(np.float32)

    matrix1 = cv2.getPerspectiveTransform(vertex, pts1)
    WIDTH, HEIGHT = 800, 1000
    
    warped_image1 = cv2.warpPerspective(frame, matrix1, (WIDTH, HEIGHT))

    # cv2.imshow('Original', frame)
    # cv2.imshow('LIned', Image)
    # cv2.imshow('Corrected', warped_image1)

    cv2.imwrite(f'D:\cv_project\original_frame\{i}.png', frame)
    cv2.imwrite(f'D:\cv_project\lined_frame\{i}.png', Image)
    cv2.imwrite(f'D:\cv_project\corrected_frame\{i}.png', warped_image1)

    i+=1
    if cv2.waitKey(0) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Not enough lines found in the image for Vanishing Point detection.
Not enough lines found in the image for Vanishing Point detection.
Not enough lines found in the image for Vanishing Point detection.
Not enough lines found in the image for Vanishing Point detection.
Not enough lines found in the image for Vanishing Point detection.
Not enough lines found in the image for Vanishing Point detection.
Not enough lines found in the image for Vanishing Point detection.
Not enough lines found in the image for Vanishing Point detection.
Not enough lines found in the image for Vanishing Point detection.
Not enough lines found in the image for Vanishing Point detection.
Not enough lines found in the image for Vanishing Point detection.
Not enough lines found in the image for Vanishing Point detection.
Not enough lines found in the image for Vanishing Point detection.
Not enough lines found in the image for Vanishing Point detection.
Not enough lines found in the image for Vanishing Point detect

C:\Users\Jatin\AppData\Local\Temp\ipykernel_29956\2541951551.py:14: RuntimeWarning: overflow encountered in int_scalars
  c = y2 - m*x2


Vanishing Point not found. Possible reason is that not enough lines are found in the image for determination of vanishing point.
Not enough lines found in the image for Vanishing Point detection.
Vanishing Point not found. Possible reason is that not enough lines are found in the image for determination of vanishing point.
Vanishing Point not found. Possible reason is that not enough lines are found in the image for determination of vanishing point.
Vanishing Point not found. Possible reason is that not enough lines are found in the image for determination of vanishing point.
Vanishing Point not found. Possible reason is that not enough lines are found in the image for determination of vanishing point.
Vanishing Point not found. Possible reason is that not enough lines are found in the image for determination of vanishing point.
Vanishing Point not found. Possible reason is that not enough lines are found in the image for determination of vanishing point.
Vanishing Point not found. Pos

: 

In [10]:
orig_folder = cwd + "\original_frame\\"
lined_folder = cwd + "\lined_frame\\"
corrected_folder = cwd + "\corrected_frame\\"

orig_folder, lined_folder, corrected_folder

('d:\\cv_project\\original_frame\\',
 'd:\\cv_project\\lined_frame\\',
 'd:\\cv_project\\corrected_frame\\')

In [11]:
orig_images = os.listdir(orig_folder)
lined_images = os.listdir(lined_folder)
correted_images =os.listdir(corrected_folder) 

In [12]:
def create_video(frame_size, images_path, video_name):
    fourcc = cv2.VideoWriter_fourcc(*"MP4V")
    video = cv2.VideoWriter(
        filename=video_name, fourcc=fourcc, fps=10.0, frameSize=frame_size
    )

    _images_list = os.listdir(images_path)
    images_list = []
    for nn in _images_list:
        images_list.append(int(nn[:-4]))
    images_list.sort()

    for i in  images_list:
        image = cv2.imread(images_path + str(i) + ".png")
        # print(image.shape)
        x = video.write(image)
        if((i+1) % 1000 == 0):
            print(i)

    video.release()

In [13]:
FRAME_SIZE = (640, 360)
create_video(FRAME_SIZE, orig_folder, "original_full.mp4")

999
1999
2999
3999
4999
5999
6999
7999
8999
9999


In [14]:
FRAME_SIZE = (640, 360)
create_video(FRAME_SIZE, lined_folder, "lined_full.mp4")

999
1999
2999
3999
4999
5999
6999
7999
8999
9999


In [15]:
FRAME_SIZE = (800, 1000)
create_video(FRAME_SIZE, corrected_folder, "corrected_full.mp4")


999
1999
2999
3999
4999
5999
6999
7999
8999
9999
